In [1]:
!pip install python-docx


  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184600 sha256=bb85df38ff9282cd7f7d501c813beafd1eae0973d614ff3c201c9a1c70f35408
  Stored in directory: c:\users\sj30153\appdata\local\pip\cache\wheels\32\b8\b2\c4c2b95765e615fe139b0b17b5ea7c0e1b6519b0a9ec8fb34d
Successfully built python-docx


In [2]:
import docx

In [3]:
def get_or_create_hyperlink_style(d):
    """If this document had no hyperlinks so far, the builtin
       Hyperlink style will likely be missing and we need to add it.
       There's no predefined value, different Word versions
       define it differently.
       This version is how Word 2019 defines it in the
       default theme, excluding a theme reference.
    """
    if "Hyperlink" not in d.styles:
        if "Default Character Font" not in d.styles:
            ds = d.styles.add_style("Default Character Font",
                                    docx.enum.style.WD_STYLE_TYPE.CHARACTER,
                                    True)
            ds.element.set(docx.oxml.shared.qn('w:default'), "1")
            ds.priority = 1
            ds.hidden = True
            ds.unhide_when_used = True
            del ds
        hs = d.styles.add_style("Hyperlink",
                                docx.enum.style.WD_STYLE_TYPE.CHARACTER,
                                True)
        hs.base_style = d.styles["Default Character Font"]
        hs.unhide_when_used = True
        hs.font.color.rgb = docx.shared.RGBColor(0x05, 0x63, 0xC1)
        hs.font.underline = True
        del hs

    return "Hyperlink"

def add_hyperlink(paragraph, link_to, text, is_external):
    ''' Adds a hyperlink within a paragraph to an internal bookmark
    or an external url '''

    part = paragraph.part

    hyperlink = docx.oxml.shared.OxmlElement('w:hyperlink')
    if is_external:
        r_id = part.relate_to(link_to,
            docx.opc.constants.RELATIONSHIP_TYPE.HYPERLINK,
            is_external= is_external)

        hyperlink.set(docx.oxml.shared.qn('r:id'), r_id, )
    else:
        hyperlink.set(docx.oxml.shared.qn('w:anchor'), link_to, )

    new_run = docx.oxml.shared.OxmlElement('w:r')
    rPr = docx.oxml.shared.OxmlElement('w:rPr')

    new_run.append(rPr)
    new_run.text = text
    new_run.style = get_or_create_hyperlink_style(part.document)

    hyperlink.append(new_run)

    paragraph._p.append(hyperlink)


def add_bookmark(paragraph, bookmark_name):
    ''' Adds a word bookmark to a run '''
    run = paragraph.add_run()
    tag = run._r
    start = docx.oxml.shared.OxmlElement('w:bookmarkStart')
    start.set(docx.oxml.ns.qn('w:id'), '0')
    start.set(docx.oxml.ns.qn('w:name'), bookmark_name)
    tag.append(start)

    text = docx.oxml.OxmlElement('w:r')
    tag.append(text)

    end = docx.oxml.shared.OxmlElement('w:bookmarkEnd')
    end.set(docx.oxml.ns.qn('w:id'), '0')
    end.set(docx.oxml.ns.qn('w:name'), bookmark_name)
    tag.append(end)

    return run

In [4]:
document = docx.Document()
document.add_heading('section 1', 2)
p1 = document.add_paragraph('Paragraph 1')

add_hyperlink(p1, 'bookmark1', 'Table 14.1.1.1.1.1', False)

p1.add_run('\n\n\n\n some text \n\n\n\n\n some text')
add_bookmark(p1, 'bookmark1')


document.save('test.docx')

In [5]:
import os
os.listdir()

['..ipynb_checkpoints',
 '.ipynb_checkpoints',
 '30153_FORM-16-FY22-23_FY 2022 - 2023..pdf',
 'Citeline changes.msg',
 'desktop.ini',
 'embedded_data_page_1.png',
 'github-recovery-codes.txt',
 'hyperlink.ipynb',
 'Study Report Body_New template.pdf',
 'test.docx',
 'test_embedded_data.docx',
 'test_embedded_data.pdf',
 'test_embedded_graph_img.docx',
 'test_embedded_graph_img.pdf',
 'ZS']